In [1]:
import os, sys, shutil
import pathlib
import glob as glob
import numpy as np
import re
import warnings
from pdbfixer import PDBFixer
from openmm.app import PDBxFile, PDBFile
import mdtraj as md

In [2]:
#warnings.filterwarnings("ignore")

In [3]:
def check_file(file):
    """
    """    
    basename = os.path.basename(file)
    #print(">{}".format(basename))

    with warnings.catch_warnings(record=True) as w:
        warnings.simplefilter("default")
        fixer = PDBFixer(filename=file)
    fixer.findMissingResidues()
    fixer.findMissingAtoms()

    
    n_atoms = 0
    n_residues = 0
    with open(file, "r") as rf:
        for l in rf.readlines():
            _l = l.strip('\n').split()
            if l.startswith("ATOM"):
                n_atoms += 1
                symbol = _l[2]
                if symbol == "P":
                    n_residues += 1


    if fixer.topology.getNumResidues() != n_residues:
        print("{}: Perhaps same residue ID? Number of residues does not match (residues: {}->{} / atoms: {}->{})".format(basename, \
                                                                                                                         n_residues, fixer.topology.getNumResidues(), \
                                                                                                                         n_atoms, fixer.topology.getNumAtoms()))
        print("-----------------------------\n")
        
        shutil.move(file, file + ".warning")
    elif fixer.missingAtoms:
        print("{}: add missing atoms. check manually".format(basename))
        print(fixer.missingAtoms)
        print("-----------------------------\n")
        fixer.addMissingAtoms()
        shutil.move(file, file + ".missing_atoms")
        PDBxFile.writeFile(fixer.topology, fixer.positions, open(file + ".missing_atoms_added", "w"))

In [4]:
if __name__ == "__main__":
    base_path = os.path.dirname(os.path.abspath("__file__")).strip('notebooks')
    
    #files_for_bpcatalog = glob.glob(os.path.join(base_path) + "data/bpcatalog/*/*.cif")
    #files_for_triplebase = glob.glob(os.path.join(base_path) + "data/triplebase/*/*.pdb")
    files_for_hairpins = glob.glob(os.path.join(base_path) + "data/HairpinLoopMotifAtlasRelease3.57/*.cif")
    files_for_internals = glob.glob(os.path.join(base_path) + "data/InternalLoopMotifAtlasRelease3.57/*.cif")
    files_for_junctions = glob.glob(os.path.join(base_path) + "data/nrlist_3.233_2.5A/JunctionLoop/*.cif")
    
    #files = files_for_bpcatalog + files_for_triplebase + files_for_hairpins + files_for_internals + files_for_junctions
    files = files_for_hairpins + files_for_internals + files_for_junctions
    
    for file in files:
        check_file(file)

HL_44730.2.cif: Perhaps same residue ID? Number of residues does not match (residues: 6->3 / atoms: 126->26)
-----------------------------



HL_51090.1.cif: add missing atoms. check manually
{<Residue 9 (A) of chain 0>: [<Atom 0 (P) of chain 0 residue 0 (A)>, <Atom 1 (OP1) of chain 0 residue 0 (A)>, <Atom 2 (OP2) of chain 0 residue 0 (A)>, <Atom 3 (O5') of chain 0 residue 0 (A)>, <Atom 4 (C5') of chain 0 residue 0 (A)>, <Atom 5 (C4') of chain 0 residue 0 (A)>, <Atom 6 (O4') of chain 0 residue 0 (A)>, <Atom 7 (C3') of chain 0 residue 0 (A)>, <Atom 8 (O3') of chain 0 residue 0 (A)>, <Atom 9 (C2') of chain 0 residue 0 (A)>, <Atom 10 (O2') of chain 0 residue 0 (A)>, <Atom 11 (C1') of chain 0 residue 0 (A)>, <Atom 15 (C5) of chain 0 residue 0 (A)>, <Atom 16 (C6) of chain 0 residue 0 (A)>, <Atom 18 (N1) of chain 0 residue 0 (A)>, <Atom 19 (C2) of chain 0 residue 0 (A)>, <Atom 20 (N3) of chain 0 residue 0 (A)>, <Atom 21 (C4) of chain 0 residue 0 (A)>]}
-----------------------------

HL_01181.4.cif: Perhaps same residue ID? Number of residues does not match (residues: 7->8 / atoms: 162->162)
-----------------------------

HL_70505.